<a href="https://colab.research.google.com/github/SandeshGitHub2077/Colab-Notebooks/blob/drive_linker/xlnet-project/xlnet_file_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get OKR train-test dataset from [here](https://github.com/SandeshGitHub2077/Colab-Notebooks/tree/main/xlnet-project/datasets) 

### Importing Libraries and data

In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [6]:
df_train = pd.read_csv("train.csv", encoding= 'unicode_escape')
df_test = pd.read_csv("test.csv", encoding= 'unicode_escape')

In [7]:
df_train = df_train.dropna(axis=1)

In [8]:
df_train.head()

,keyresult,target
0,Exceed Net Promoter Score (NPS) of over 8.0,1
1,Get 1000 survey responses to annual satisfact...,1
2,Conduct 50 phone interviews with top customers,1
3,Conduct 15 phone interviews with recently chu...,1
4,Present an action plan of 10 improvements for...,1


In [9]:
df_train.target.value_counts()

1    537
0    304
Name: target, dtype: int64

In [10]:
df_train.sample(10)['keyresult'].tolist()

['some of our existing customers try it',
 '10% of our existing customers try it',
 'to deliver 5 models to a business unit on time the models were requested by and the requirements approved with the business unit.',
 ' Integrate user testing into pre-launch testing phase\xa0',
 'our vacancies web page visits increase from 600 to 900',
 ' Support team to conduct 50 phone interviews with churned accounts\xa0',
 ' Apply for 5 external design competitions\xa0',
 'do few analyst briefings ',
 'each support rep to maintain a personal csat',
 ' Conduct 3 monthly designer meetups at our office\xa0']

### Cleaning
 - Remove unnecessary grammar

In [11]:
! pip install tweet-preprocessor
import preprocessor as p
import re
# def clean_text(text):
#   text = text.replace("#","")
#   return p.clean(text)

def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

df_train['clean_text'] = df_train['keyresult'].astype(str).progress_map(clean_text)
df_test['clean_text'] = df_test['keyresult'].astype(str).progress_map(clean_text)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
# splitting the data into training and test dataset
X = df_train['clean_text']
y = df_train['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

In [15]:
train_df.shape, eval_df.shape

((672, 2), (169, 2))

In [16]:
# transformers - SOTA implementation of pretrained models
!pip install -U simpletransformers 

     |████████████████████████████████| 225kB 8.1MB/s 
     |████████████████████████████████| 7.8MB 40.4MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 266kB 53.1MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 133kB 52.9MB/s 
     |████████████████████████████████| 1.2MB 40.1MB/s 
     |████████████████████████████████| 2.5MB 45.0MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
     |████████████████████████████████| 1.8MB 45.6MB/s 
     |████████████████████████████████| 174kB 58.6MB/s 
     |████████████████████████████████| 4.2MB 34.0MB/s 
     |████████████████████████████████| 112kB 54.6MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 245kB 57.7MB/s 
     |████████████████████████████████| 122kB 57.1MB/s 
     |████████████████████████████████| 901kB 42.9MB/s 
     |████████████████████████████████| 102kB 15.2MB/s

In [17]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 32,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 6,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}



# ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', args={'num_train_epochs':6, 'train_batch_size':32, 'max_seq_length':128})

# Model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

INFO:filelock:Lock 139622287319952 acquired on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139622287319952 released on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139622227621520 acquired on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


INFO:filelock:Lock 139622227621520 released on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_xlnet_128_2_2


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarnin

INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlnet_128_2_2


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9626262543456602, 'tp': 105, 'tn': 61, 'fp': 0, 'fn': 3, 'auroc': 0.9948391013964785, 'auprc': 0.9973852754406616, 'acc': 0.9822485207100592, 'eval_loss': 0.08155961334705353}


In [18]:
result

{'acc': 0.9822485207100592,
 'auprc': 0.9973852754406616,
 'auroc': 0.9948391013964785,
 'eval_loss': 0.08155961334705353,
 'fn': 3,
 'fp': 0,
 'mcc': 0.9626262543456602,
 'tn': 61,
 'tp': 105}

In [19]:
predictions, raw_outputs = model.predict(df_test.clean_text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


### Saving model on a local file to skip the execution time next time we predict

In [20]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

Loading model

In [21]:
loaded_model = pickle.load(open(filename, 'rb'))

### Lets Put the Input user and its processing in a function

In [22]:
def singleInput():
  userText = input("Enter KR:\n")
  userText = clean_text(userText)
  predictions1 = loaded_model.predict(userText)
  print('Measurable' if predictions1[0][0] == 1 else 'Non-Measurable')

In [23]:
singleInput()

Enter KR:
asfdas


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.



Non-Measurable


### Saving model on drive

In [ ]:
import torch
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'xlnet_model.pt'
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model, path)

### Loading the model from drive
* cutting the time of re running the model

In [1]:
!pip install -U simpletransformers
!pip install tweet-preprocessor
import pandas as pd
import numpy as np
import preprocessor as p
import re
from tqdm.notebook import tqdm
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn
import torch
from google.colab import drive
def clean_text(text):
  text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
  text = re.sub('\t', ' ',  text)
  text = re.sub(r" +", ' ', text)
  return text

     |████████████████████████████████| 225kB 4.3MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 3.3MB 14.3MB/s 
     |████████████████████████████████| 133kB 34.9MB/s 
     |████████████████████████████████| 7.8MB 31.2MB/s 
     |████████████████████████████████| 2.5MB 35.4MB/s 
     |████████████████████████████████| 1.2MB 27.2MB/s 
     |████████████████████████████████| 1.8MB 31.9MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 266kB 36.2MB/s 
     |████████████████████████████████| 174kB 39.7MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 112kB 41.7MB/s 
     |████████████████████████████████| 4.2MB 33.2MB/s 
     |████████████████████████████████| 901kB 29.1MB/s 
     |████████████████████████████████| 143kB 41.1MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 245kB 36.6MB/s 

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
model_save_name = 'xlnet_model.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
driveModel = torch.load(path, map_location=torch.device('cpu'))

In [4]:
def singleInput1():
  userText1 = input("Enter KR:\n")
  userText1 = clean_text(userText1)
  predictions2 = driveModel.predict(userText1)
  print('Measurable' if predictions2[0][0] == 1 else 'Non-Measurable')
singleInput1()

Enter KR:
loca works only on weekdays


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Non-Measurable
